In [8]:
import yfinance as yf
from datetime import date,timedelta
from pycaret import time_series as ts
from pycaret.time_series import *
import plotly.express as px
import warnings
import pandas as pd

In [2]:
warnings.filterwarnings('ignore')

In [3]:
stock_name = ["AKBNK.IS"]
yday = date.today() - timedelta(days=1)
startday = yday - timedelta(days=729)
stock_data = yf.download(tickers = stock_name[0], start = startday, end = yday)
df = stock_data.copy()

In [33]:
stock_data.reset_index(inplace=True)
stock_data['MA30'] = stock_data['Close'].rolling(30).mean()
stock_price_chart = px.line(stock_data,
                           x='Date',
                           y=['Close','MA30'],
                           title=f'{stock_name[0]} Last 2 Year Daily Close Price', 
                           template = 'plotly_dark',
                           color_discrete_map={'Close':'green', 'MA30':'red'}, 
                           width=800, height=800)
stock_price_chart.show()

In [10]:
start_date = df.index.min()
end_date = df.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
complete_df = pd.DataFrame(index=date_range)
#Create missing days (holiday, weekends etc.) and linear fill
merged_df = complete_df.merge(df, left_index=True, right_index=True, how='left')
# Maybe time, quadratic or from_derivatives
filled_df = merged_df.interpolate(method='linear')
fig_kwargs={'renderer': 'notebook'}
forecast_horizon = 15
fold = 3
exp_name = setup(data=filled_df['Close'], fh=forecast_horizon, fig_kwargs=fig_kwargs, session_id=73, verbose=False)
selected_models = compare_models(n_select=3,verbose=False)
blended = blend_models(selected_models,verbose=False)
tuneds = [tune_model(model,verbose=False) for model in selected_models]
selected_models.append(blended)
[selected_models.append(model) for model in tuneds]
model_metrics = []
model_names = []
for i,model in enumerate(selected_models):
    _ = exp_name.predict_model(model,verbose=False)
    model_names.append(model)
    model_metrics.append(exp_name.pull())
mase_vals = [model_metrics[i].MASE.values[0] for i in range(len(model_metrics))]
models_df = pd.DataFrame()
models_df["Models"] = model_names
models_df["MASE"] = mase_vals
sorted_df = models_df.sort_values(by=["MASE"])
best_model = sorted_df["Models"].iloc[0]
finalized_best_model = exp_name.finalize_model(best_model)
save_model(finalized_best_model, 'finalized_best_model', verbose=False)

,Description,Value
0,session_id,123
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(726, 1)"
5,Transformed data shape,"(726, 1)"
6,Transformed train set shape,"(666, 1)"
7,Transformed test set shape,"(60, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,5.0063,3.6453,1.1480,1.4925,0.0679,0.0659,-0.3656,0.6267
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,6.5719,4.4758,1.4915,1.8240,0.0854,0.0867,-0.9896,0.2733
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,7.0838,4.7837,1.6020,1.9409,0.0931,0.0928,-1.0689,0.2600
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,7.7489,5.2214,1.7481,2.1141,0.0987,0.1048,-1.4437,0.2400
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,7.8714,5.2632,1.8076,2.1515,0.1074,0.1013,-1.8967,0.9100
snaive,Seasonal Naive Forecaster,8.6943,5.7723,1.9556,2.3354,0.1136,0.1138,-1.9204,0.0200
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,8.3305,5.5358,1.9154,2.2642,0.1141,0.1064,-2.2976,0.4000
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,8.6551,5.7593,1.9666,2.3389,0.1158,0.1110,-2.1613,0.3367
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,8.7253,5.7320,1.9734,2.3233,0.1159,0.1129,-1.9893,0.2833
naive,Naive Forecaster,8.8936,5.8548,1.9953,2.3622,0.1164,0.1155,-1.9526,0.0133
